In [1]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from prophet import Prophet
import re
import copy

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [3]:
drive_path = "/content/drive/MyDrive/데이터 경진대회/제주 특산물 가격 예측"
train_data_path = drive_path + "/train.csv"
test_data_path = drive_path + "/test.csv"
sample_submission_path = drive_path + "/sample_submission.csv"

In [4]:
train_df = pd.read_csv(train_data_path)
test_df = pd.read_csv(test_data_path)

In [5]:
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])

In [6]:
category_list = train_df["ID"].str.replace(r'_\d{8}$', '', regex=True).unique()

In [7]:
def handle_data_to_prophet(df) :
  transformed_df = df[["ID", "timestamp", "price(원/kg)"]]
  transformed_df["ID"] = transformed_df["ID"].str.replace(r'_\d{8}$', '', regex=True)
  transformed_df = transformed_df.rename(columns={'timestamp': 'ds', 'price(원/kg)': 'y'})
  return transformed_df.copy()

In [8]:
prophet_train_data = handle_data_to_prophet(train_df)

In [9]:
prophet_train_data.head(5)

,ID,ds,y
0,TG_A_J,2019-01-01,0.0
1,TG_A_J,2019-01-02,0.0
2,TG_A_J,2019-01-03,1728.0
3,TG_A_J,2019-01-04,1408.0
4,TG_A_J,2019-01-05,1250.0


### Prophet 하이퍼 파라미터 탐색

In [10]:
search_data = prophet_train_data[prophet_train_data["ID"] == "TG_A_J"].drop(columns=["ID"]).sort_values("ds").copy()

In [51]:
train_size = int(len(search_data) * 0.8)
search_train_data = search_data[:train_size]
# 만약 0을 제외하고 사용하려면 아래 줄 주석을 해제하세요
# search_train_data = search_train_data[search_train_data["y"] > 0]
search_test_data = search_data[train_size:]

In [52]:
search_train_data

,ds,y
2,2019-01-03,1728.0
3,2019-01-04,1408.0
4,2019-01-05,1250.0
6,2019-01-07,1474.0
7,2019-01-08,1326.0
...,...,...
1212,2022-04-27,4915.0
1213,2022-04-28,5508.0
1214,2022-04-29,5998.0
1215,2022-04-30,5415.0


In [53]:
param_grid = {
    'growth': ['linear'],
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 1, 5, 10],
    'seasonality_prior_scale': [0.01, 0.1, 1, 5, 10],
    'changepoint_range': [0.8, 0.85, 0.9, 0.95],
    'yearly_seasonality': [True, False],
    'weekly_seasonality': [True, False],
    'daily_seasonality': [True, False],
    'holidays_prior_scale': [0.1, 1, 5],
}

In [54]:
def prophet_grid_search(num_iterations, param_grid, train_data, y):
  results = []

  for _ in range(num_iterations) :
    random_params = {
        key : random.choice(value) for key, value in param_grid.items()
    }

    current_model = Prophet(**random_params)
    current_model.fit(train_data)

    y_size = len(y)

    future = current_model.make_future_dataframe(periods=y_size)
    forecast = current_model.predict(future)
    m_predict = forecast["yhat"].values[-y_size:]

    se = np.square(m_predict - y)
    rmse = np.sqrt(np.mean(se))

    results.append((random_params, rmse))
  return results


In [55]:
y = search_test_data["y"].values[:]
results = prophet_grid_search(30, param_grid, search_train_data, y)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpsannuvhc/y6v48u4y.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsannuvhc/b71i7fy4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=10231', 'data', 'file=/tmp/tmpsannuvhc/y6v48u4y.json', 'init=/tmp/tmpsannuvhc/b71i7fy4.json', 'output', 'file=/tmp/tmpsannuvhc/prophet_modelnvdj1shc/prophet_model-20231108091431.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:14:31 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:14:31 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsannuvhc/eoa_85ay.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsannuvhc/e7w25y1t.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [56]:
sorted_results = sorted(results, key=lambda x : x[1])

In [57]:
sorted_results[:5]

[({'growth': 'linear',
   'changepoint_prior_scale': 5,
   'seasonality_prior_scale': 5,
   'changepoint_range': 0.8,
   'yearly_seasonality': True,
   'weekly_seasonality': True,
   'daily_seasonality': True,
   'holidays_prior_scale': 5},
  2891.5523557453103),
 ({'growth': 'linear',
   'changepoint_prior_scale': 5,
   'seasonality_prior_scale': 0.1,
   'changepoint_range': 0.85,
   'yearly_seasonality': True,
   'weekly_seasonality': True,
   'daily_seasonality': False,
   'holidays_prior_scale': 0.1},
  2989.3289018390233),
 ({'growth': 'linear',
   'changepoint_prior_scale': 5,
   'seasonality_prior_scale': 1,
   'changepoint_range': 0.8,
   'yearly_seasonality': True,
   'weekly_seasonality': False,
   'daily_seasonality': False,
   'holidays_prior_scale': 5},
  3032.868415119233),
 ({'growth': 'linear',
   'changepoint_prior_scale': 0.1,
   'seasonality_prior_scale': 1,
   'changepoint_range': 0.95,
   'yearly_seasonality': False,
   'weekly_seasonality': True,
   'daily_seasona

In [58]:
best_params, best_rmse = min(results, key=lambda x: x[1])

In [59]:
best_params, best_rmse

({'growth': 'linear',
  'changepoint_prior_scale': 5,
  'seasonality_prior_scale': 5,
  'changepoint_range': 0.8,
  'yearly_seasonality': True,
  'weekly_seasonality': True,
  'daily_seasonality': True,
  'holidays_prior_scale': 5},
 2891.5523557453103)

In [ ]:
test_model = Prophet(**best_params)
test_model.fit(search_train_data)
future = test_model.make_future_dataframe(periods=305)
forecast = test_model.predict(future)
test_model.plot(forecast)

In [19]:
prophet_map = {}

In [20]:
for category in category_list :
  main_model = Prophet(
      **best_params
  )
  current_category_train_x = prophet_train_data[prophet_train_data["ID"] == category].drop(columns=["ID"]).sort_values("ds")
  prophet_map[category] = main_model.fit(current_category_train_x)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpsannuvhc/qklmx162.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsannuvhc/1pbi5gf_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=77545', 'data', 'file=/tmp/tmpsannuvhc/qklmx162.json', 'init=/tmp/tmpsannuvhc/1pbi5gf_.json', 'output', 'file=/tmp/tmpsannuvhc/prophet_modelxlbxo0ls/prophet_model-20231108090014.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:00:14 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:00:14 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsannuvhc/g_pydrcx.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsannuvhc/qowf_48h.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

In [21]:
test_df["price(원/kg)"] = 0

In [22]:
prophet_test_data = handle_data_to_prophet(test_df)

In [23]:
result_map = {}
for category in category_list :
  current_category_test_x = prophet_test_data[prophet_test_data["ID"] == category].drop(columns=["ID"])
  model = prophet_map[category]
  y = model.predict(current_category_test_x)
  result_map[category] = y["yhat"].values.tolist()

In [24]:
copy_result_map = copy.deepcopy(result_map)

In [25]:
def assign_result(row) :
  category = re.sub(r'_\d{8}$', '', row["ID"])
  result = copy_result_map[category]
  return result.pop(0)

In [26]:
submission = pd.read_csv(sample_submission_path)

In [27]:
submission.head()

,ID,answer
0,TG_A_J_20230304,0
1,TG_A_J_20230305,0
2,TG_A_J_20230306,0
3,TG_A_J_20230307,0
4,TG_A_J_20230308,0


In [28]:
submission["answer"] = 0

In [29]:
submission["answer"] = submission.apply(assign_result, axis=1)

In [30]:
submission.tail()

,ID,answer
1087,RD_F_J_20230327,365.668322
1088,RD_F_J_20230328,371.967115
1089,RD_F_J_20230329,363.941160
1090,RD_F_J_20230330,357.875728
1091,RD_F_J_20230331,353.250750


In [31]:
submission[submission["answer"] < 0]

,ID,answer
400,CR_D_S_20230312,-0.470456
403,CR_D_S_20230315,-3.445713
405,CR_D_S_20230317,-1.136784
406,CR_D_S_20230318,-6.061351
407,CR_D_S_20230319,-6.800384
408,CR_D_S_20230320,-0.002742
409,CR_D_S_20230321,-1.122018
410,CR_D_S_20230322,-8.599626
412,CR_D_S_20230324,-4.994692
413,CR_D_S_20230325,-9.190615


In [32]:
submission.to_csv(drive_path + "/submission_231108_1.csv", index=False)